In [1]:
FILE = 'almaty_small'

In [2]:
class TNode:
    def __init__(self, id, lat, lng):
        self.id = id
        self.lat = lat
        self.lng = lng

class TGraph:
    node_id_to_index = {}
    nodes = []
    edges: list[set] = []
    edges_list: list[list] = []

    def __init__(self) -> None:
        self.node_id_to_index = {}
        self.nodes = []
        self.edges = []
        self.edges_list = []

    def add_node(self, node: TNode):
        if node.id in self.node_id_to_index:
            raise Exception("Node already exists")
        index = len(self.nodes)
        self.node_id_to_index[node.id] = index
        self.nodes.append(node)
        self.edges.append(set())

    def add_edge(self, src_id, dst_id):
        if src_id not in self.node_id_to_index:
            return
            # raise Exception("Source node does not exist")
        if dst_id not in self.node_id_to_index:
            return
            # raise Exception("Destination node does not exist")
        
        src_index = self.node_id_to_index[src_id]
        dst_index = self.node_id_to_index[dst_id]
        self.edges[src_index].add(dst_index)
        self.edges[dst_index].add(src_index)
        self.edges_list.append([src_index, dst_index])

    def __str__(self) -> str:
        return f"{len(self.nodes)} nodes, {len(self.edges_list)}"

In [3]:
import json 
from pprint import pprint

def read_graph_from_json(file_path: str) -> TGraph:
    with open(file_path, 'r') as file:
        data = json.load(file)
        graph = TGraph()
        for element in data['elements']:
            if element['type'] == 'node':
                graph.add_node(TNode(element['id'], element['lat'], element['lon']))
        
        for element in data['elements']:
            if element['type'] == 'way':
                for i in range(len(element['nodes']) - 1):
                    src = element['nodes'][i]
                    dst = element['nodes'][i + 1]
                    graph.add_edge(src, dst)

        return graph
    
graph = read_graph_from_json(f"../data_samples/{FILE}.json")

print(graph)

24218 nodes, 25314


In [4]:
from planegeometry.structures.segments import Segment, Point
from planegeometry.algorithms.bentleyottmann2 import BentleyOttmann
from planegeometry.algorithms.geomtools import find_intersection_points
from linesegmentintersections import bentley_ottman

segment_list = list()

MOVE = 0.000001

for src_index in range(len(graph.nodes)):
    src = graph.nodes[src_index]
    for dst_index in graph.edges[src_index]:
        if src_index >= dst_index:
            continue
            
        dst = graph.nodes[dst_index]
        
        # print(src.lat, src.lng, dst.lat, dst.lng)

        p1 = Point(src.lat, src.lng)
        p2 = Point(dst.lat, dst.lng)
        seg = Segment(p1, p2)
        
        # Need to move endpoints to center by a small amount.
        center = seg.center()
        dir1 = p1 - center
        new_p1 = p1 - dir1 * (1 / dir1.__abs__()) * MOVE

        dir2 = p2 - center
        new_p2 = p2 - dir2 * (1 / dir2.__abs__()) * MOVE

        segment_list.append([[new_p1.x, new_p1.y], [new_p2.x, new_p2.y]])

li = bentley_ottman(segment_list)

print(li)

[(43.19455203856975, 76.89424664917124), (43.194569228532345, 76.89438539903881), (43.19463112160696, 76.88144090533842), (43.19463800064841, 76.88236208085891), (43.19474290596647, 76.89424610734488), (43.19475939323326, 76.89438158686204), (43.19476195660676, 76.88146702759605), (43.194772190723555, 76.88233809188432), (43.20014046425235, 76.86901289541953), (43.20016907056969, 76.86912471139397), (43.200291495535716, 76.86892355884997), (43.20032890601231, 76.86902756599696), (43.20076582199291, 76.86864708766717), (43.200796544763506, 76.86875960314188), (43.20080686242605, 76.9055504712023), (43.20081227600684, 76.90554602819122), (43.200875558697184, 76.9056965057842), (43.200933217227714, 76.90544676982515), (43.20099491243069, 76.90559561653072), (43.20100155794489, 76.90558999910303), (43.207884394260425, 76.86328856032844), (43.20792903461932, 76.86340592588147), (43.20797085835737, 76.86317561633572), (43.20801315328303, 76.86329581111686), (43.208034567814984, 76.8630923956

In [5]:
for i in range(len(li)):
    # i = list(inter)
    inter = li[i]
    print(inter.x, inter.y, i)

43.19455203856975 76.89424664917124 0
43.194569228532345 76.89438539903881 1
43.19463112160696 76.88144090533842 2
43.19463800064841 76.88236208085891 3
43.19474290596647 76.89424610734488 4
43.19475939323326 76.89438158686204 5
43.19476195660676 76.88146702759605 6
43.194772190723555 76.88233809188432 7
43.20014046425235 76.86901289541953 8
43.20016907056969 76.86912471139397 9
43.200291495535716 76.86892355884997 10
43.20032890601231 76.86902756599696 11
43.20076582199291 76.86864708766717 12
43.200796544763506 76.86875960314188 13
43.20080686242605 76.9055504712023 14
43.20081227600684 76.90554602819122 15
43.200875558697184 76.9056965057842 16
43.200933217227714 76.90544676982515 17
43.20099491243069 76.90559561653072 18
43.20100155794489 76.90558999910303 19
43.207884394260425 76.86328856032844 20
43.20792903461932 76.86340592588147 21
43.20797085835737 76.86317561633572 22
43.20801315328303 76.86329581111686 23
43.208034567814984 76.86309239566067 24
43.20807651986206 76.86321286

In [6]:
from planegeometry.structures.segments import Segment, Point
from planegeometry.algorithms.bentleyottmann2 import BentleyOttmann
from planegeometry.algorithms.geomtools import find_intersection_points
from linesegmentintersections import bentley_ottman

segment_list = list()

MOVE = 0.000001

edges_to_remove = []

for inter in li:
    inter_point = Point(inter.x, inter.y)
    for src_index in range(len(graph.nodes)):
        src = graph.nodes[src_index]
        for dst_index in graph.edges[src_index]:
            if src_index >= dst_index:
                continue
                
            dst = graph.nodes[dst_index]
            
            # print(src.lat, src.lng, dst.lat, dst.lng)

            p1 = Point(src.lat, src.lng)
            p2 = Point(dst.lat, dst.lng)
            seg = Segment(p1, p2)
            
            if min(p1.x, p2.x) <= inter_point.x and inter_point.x <= max(p1.x, p2.x) and (abs(seg.calculate_y(inter_point.x) - inter_point.y) < 0.00001):
                # print(seg, inter_point)
                edges_to_remove.append((src_index, dst_index))

for edge in edges_to_remove:
    src_index, dst_index = edge
    if dst_index in graph.edges[src_index]:
        graph.edges[src_index].remove(dst_index)
    if src_index in graph.edges[dst_index]:
        graph.edges[dst_index].remove(src_index)

In [7]:
# Format to graph to json format

data = {
    "elements": []
}

for node in graph.nodes:
    data['elements'].append({
        "type": "node",
        "id": node.id,
        "lat": node.lat,
        "lon": node.lng
    })

for src_index in range(len(graph.nodes)):
    for dst_index in graph.edges[src_index]:
        src = graph.nodes[src_index]
        dst = graph.nodes[dst_index]
        data['elements'].append({
            "type": "way",
            "nodes": [src.id, dst.id]
        })

with open(f'../data_samples/{FILE}_cleaned.json', 'w') as file:
    json.dump(data, file)